In [1]:
# prepare data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
import glob
import operator
from IPython.display import display, HTML

In [2]:
%load_ext autoreload
%autoreload 1

from shared_globals_functions import *
%aimport shared_globals_functions

In [3]:
t_name='t1'
merge_list=['t1.1', 't1.2', 't1.3', 't1.4']

cust_dir = 'test'

tdidf_max_df=1.0 # defaults to 1.0 (float), can be an int too
tdidf_min_df=4
use_tfidfvectorizer = True
use_stopwords = True
use_extended_stopwords = False

umap_metric='hellinger'
umap_ncomponents=2
umap_random_state=42
umap_min_dist=0 #0.1
umap_n_neighbors=30 #10, 30

min_cluster=10
max_cluster=30
clust_int=3

ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50

#tf-idf vectorizer
#max_df=0.3
#min_df=

#kmeans
#n_clusters
#n_init
#max_iter

In [4]:
# Parameters
cust_dir = "ar1/"
use_tfidfvectorizer = True
tdidf_max_df = 1.0
tdidf_min_df = 4
use_stopwords = True
use_extended_stopwords = False
umap_metric = "hellinger"
umap_ncomponents = 2
umap_random_state = 42
umap_min_dist = 0
umap_n_neighbors = 30
min_cluster = 10
max_cluster = 30
clust_int = 3
ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50
t_name = "t2.5.1s"
merge_list = ["t2.5.2", "t2.5.4", "t2.5.6"]


In [5]:
testset = from_disk(t_name)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.5.1s.pkl


In [6]:
len(testset[testset['topics'].str.len() > 0])

218378

In [7]:
if len(merge_list)>0:
    t_name = f'{t_name}+{"+".join([re.sub(r"t", "", tid) for tid in merge_list])}'
    print(t_name)

t2.5.1s+2.5.2+2.5.4+2.5.6


In [8]:
testset_merge = create_merge(testset, merge_list, merge_to_one=True)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.5.2.pkl


reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.5.4.pkl


reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.5.6.pkl


In [9]:
len(testset_merge[testset_merge['topics'].str.len() > 0])

218378

In [10]:
df = testset_merge.merge(testset, on=['name'], how='left', indicator=True)

In [11]:
df[df['_merge']!='both']

,name,topics_x,topics_y,_merge


In [12]:
df[df['topics_x'].str.len() == 0]

,name,topics_x,topics_y,_merge


In [13]:
if len(merge_list)>0:
    testset = testset_merge

In [14]:
import pandas as pd
import umap
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
orig_testset = testset

In [16]:
my_stop_words=None
if use_stopwords:
    my_stop_words='english'
    if use_extended_stopwords:
        print("todo")

if use_tfidfvectorizer:
    vectorizer = TfidfVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)
else:
    vectorizer = CountVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)

print(f"total repos {len(testset[testset['topics'].str.len() > 0])}")
testset = testset[testset['topics'].str.len() > 0].reset_index()

if isinstance(testset['topics'][0], list):
    testset['topics'] = testset['topics'].apply(lambda x: ' '.join(x))
    print("topics came as list ... consider joining it before")

cr_start()
word_doc_matrix = vectorizer.fit_transform(testset['topics'])
runtime_vectorizer = cr_getrt()

total repos 218378


topics came as list ... consider joining it before


In [17]:
runtime_vectorizer

17.6684889793396

In [18]:
word_doc_matrix

<218378x9599 sparse matrix of type '<class 'numpy.float64'>'
	with 35837383 stored elements in Compressed Sparse Row format>

In [19]:
class ProgressWriter:
    def write(self, text):
        print("custom progress", n, total)

    def flush(self):
        pass

#tqdm_kwds = {"file": "progress_writer", "disable": False }
tqdm_kwds = {"disable": False }   


In [20]:
%%time
# try also with default parameters for n_neighbors and min_dist

cr_start()

embedding = umap.UMAP(
    n_neighbors=umap_n_neighbors,
    min_dist=umap_min_dist,
    n_components=umap_ncomponents, metric=umap_metric,
    tqdm_kwds=tqdm_kwds,
    random_state=umap_random_state,
).fit(word_doc_matrix)

runtime_umap = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/umap/umap_.py:125: UserWarning: A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 249 edges.
It has only fully disconnected 5 vertices.
Use umap.utils.disconnected_vertices() to identify them.
  warn(


Epochs completed:   0%|            0/200 [00:00]

CPU times: user 1h 46min 47s, sys: 5min 19s, total: 1h 52min 7s
Wall time: 21min 39s


In [21]:
print(runtime_umap)

1299.0806198120117


In [22]:
%%time

cr_start()
clusterable_embedding = embedding.transform(word_doc_matrix)
runtime_umap_transform = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Epochs completed:   0%|            0/30 [00:00]

CPU times: user 3h 39min 44s, sys: 10.9 s, total: 3h 39min 55s
Wall time: 3h 36min 16s


In [23]:
print(runtime_umap_transform)

12976.100148200989


In [24]:
umap_embedding = pd.DataFrame(embedding.embedding_, columns=['x', 'y']) # (umap_t)
umap_clusterable_embedding = pd.DataFrame(clusterable_embedding, columns=['x', 'y']) # (umap_t)

umap_embedding['name'] = testset['name']
umap_clusterable_embedding['name'] = testset['name']

to_disk_results(umap_embedding, f'umap_embedding_{t_name}', subdir=f"{cust_dir}")
to_disk_results(umap_clusterable_embedding, f'umap_clusterable_embedding_{t_name}', subdir=f"{cust_dir}")

writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//umap_embedding_t2.5.1s+2.5.2+2.5.4+2.5.6.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//umap_clusterable_embedding_t2.5.1s+2.5.2+2.5.4+2.5.6.pkl


https://hdbscan.readthedocs.io/en/latest/parameter_selection.html

Since we have seen that min_samples clearly has a dramatic effect on clustering, the question becomes: how do we select this parameter? The simplest intuition for what min_samples does is provide a measure of how conservative you want you clustering to be. The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas. We can see this in practice by leaving the min_cluster_size at 60, but reducing min_samples to 1.

In [25]:
import hdbscan

def clusterapply_hdbscan(hdbscan_min_samples, min_cluster_size, runner_name):
    cr_start()
    labels = hdbscan.HDBSCAN(
        min_samples=hdbscan_min_samples,
        min_cluster_size=min_cluster_size,
    ).fit_predict(clusterable_embedding)
    runtime_hdb = cr_getrt()
    
    stats = {'type': "hdb",
             'testset': t_name,
             'version': td_version,
            }
    
    detail_stats = {'hdb_minclust': min_cluster_size,
                    'hdb_minsamp': hdbscan_min_samples,
                    'runtime': runtime_hdb,
                    'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                    'umap': f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}",
                    #'gtr': grtr,
                    #'gtr_rel': grtr_rt
                   }
    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'tot_anom':no_anomalies, 'no_lab':missing_clabel, 'anom':identified_anom})
    prediction.update(detail_stats)
    res.append(prediction.copy())
    display(pd.DataFrame.from_dict(res))
    return labels

res = []
hdb_labels_results = pd.DataFrame()
hdb_labels_results['name'] = testset['name'].copy()


for cs in range(cs_min, cs_max, cs_int): #50, 500
    for ss in range(ss_min, ss_max, ss_int): #50 500
        runner_name = f'ss{ss}cs{cs}'
        hdb_labels_results[runner_name] = clusterapply_hdbscan(ss, cs, runner_name)

results = pd.DataFrame.from_dict(res)
res = []


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss50cs50,50,50,4.860882,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss50cs50,50,50,4.860882,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss100cs50,50,100,4.815160,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss50cs50,50,50,4.860882,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss100cs50,50,100,4.815160,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss150cs50,50,150,5.244986,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss50cs50,50,50,4.860882,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss100cs50,50,100,4.815160,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss150cs50,50,150,5.244986,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss200cs50,50,200,5.868094,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss50cs50,50,50,4.860882,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss100cs50,50,100,4.815160,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss150cs50,50,150,5.244986,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss200cs50,50,200,5.868094,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
4,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss250cs50,50,250,6.535496,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."


...

,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss50cs50,50,50,4.860882,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss100cs50,50,100,4.815160,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss150cs50,50,150,5.244986,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss200cs50,50,200,5.868094,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
4,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss250cs50,50,250,6.535496,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
...,...,...,...,...,...,...,...,...,...
72,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss50cs450,450,50,4.418029,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
73,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss100cs450,450,100,4.923901,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
74,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss150cs450,450,150,5.401174,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
75,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss200cs450,450,200,6.073345,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss50cs50,50,50,4.860882,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss100cs50,50,100,4.815160,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss150cs50,50,150,5.244986,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss200cs50,50,200,5.868094,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
4,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss250cs50,50,250,6.535496,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
...,...,...,...,...,...,...,...,...,...
73,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss100cs450,450,100,4.923901,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
74,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss150cs450,450,150,5.401174,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
75,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss200cs450,450,200,6.073345,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
76,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss250cs450,450,250,6.635625,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss50cs50,50,50,4.860882,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss100cs50,50,100,4.815160,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss150cs50,50,150,5.244986,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss200cs50,50,200,5.868094,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
4,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss250cs50,50,250,6.535496,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
...,...,...,...,...,...,...,...,...,...
74,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss150cs450,450,150,5.401174,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
75,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss200cs450,450,200,6.073345,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
76,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss250cs450,450,250,6.635625,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
77,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss300cs450,450,300,7.220983,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss50cs50,50,50,4.860882,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss100cs50,50,100,4.815160,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss150cs50,50,150,5.244986,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss200cs50,50,200,5.868094,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
4,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss250cs50,50,250,6.535496,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
...,...,...,...,...,...,...,...,...,...
75,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss200cs450,450,200,6.073345,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
76,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss250cs450,450,250,6.635625,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
77,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss300cs450,450,300,7.220983,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
78,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss350cs450,450,350,7.889423,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss50cs50,50,50,4.860882,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
1,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss100cs50,50,100,4.815160,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
2,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss150cs50,50,150,5.244986,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
3,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss200cs50,50,200,5.868094,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
4,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss250cs50,50,250,6.535496,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
...,...,...,...,...,...,...,...,...,...
76,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss250cs450,450,250,6.635625,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
77,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss300cs450,450,300,7.220983,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
78,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss350cs450,450,350,7.889423,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."
79,hdb,t2.5.1s+2.5.2+2.5.4+2.5.6,1,ss400cs450,450,400,8.496258,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...","hellinger, rt_umap_tf: 12976.100148200989, rt_..."


In [26]:
hdb_labels_results

,name,ss50cs50,ss100cs50,ss150cs50,ss200cs50,ss250cs50,ss300cs50,ss350cs50,ss400cs50,ss450cs50,...,ss450cs400,ss50cs450,ss100cs450,ss150cs450,ss200cs450,ss250cs450,ss300cs450,ss350cs450,ss400cs450,ss450cs450
0,technopagan/mss-saliency,58,27,20,10,8,6,6,3,-1,...,-1,3,3,3,3,-1,-1,-1,-1,-1
1,snmnmin12/selective-search,254,29,23,12,11,7,7,4,3,...,2,4,4,4,4,4,2,3,2,2
2,google/chaps-linux,-1,-1,-1,-1,10,48,-1,6,6,...,4,-1,5,-1,6,3,4,-1,4,4
3,lukasolson/speech-router,249,-1,-1,-1,10,33,43,6,6,...,4,24,5,-1,6,3,4,24,4,4
4,XavierTecBr/qbr-admin,189,92,27,57,10,39,30,6,6,...,4,18,5,5,6,3,4,18,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218373,bhagatabhijeet/bhagatabhijeet,55,-1,-1,4,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,-1,-1,-1
218374,LandGrey/SpringBootVulExploit,-1,39,33,-1,10,-1,-1,6,6,...,4,-1,5,-1,6,3,4,-1,4,4
218375,mucomplex/Pentest_checklist,-1,101,88,64,10,38,34,6,6,...,4,18,5,20,6,3,4,18,4,4
218376,bigsen/Vision_Track,-1,29,23,12,11,7,7,4,3,...,2,4,4,4,4,4,2,3,2,2


In [27]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/hdb_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'hdb_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'hdb_results', subdir=f"{cust_dir}")
to_disk_results(hdb_labels_results, f'hdb_labels_results_{t_name}', subdir=f"{cust_dir}")

saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//hdb_labels_results_t2.5.1s+2.5.2+2.5.4+2.5.6.pkl


In [28]:

umap_embedding['topics'] = testset['topics']
umap_clusterable_embedding['topics'] = testset['topics']

In [29]:
import sklearn.cluster as cluster

def clusterapply_kmeans(testset, clustersize, runner_name, opt_comment=""):
    # Perform k-means clustering
    
    kmeans = cluster.KMeans(n_clusters=clustersize, random_state=42)
        
    if('y' in testset.columns):
        #print("assuming umap reduced testset here")
        testset = testset.copy()
        dropping_not_connected = len(testset[testset['y'].isnull()])
        testset = testset[testset['y'].notna()]
        cr_start()
        kmeans.fit(testset[['x', 'y']].to_numpy())
        runtime_kmeans = cr_getrt()
        opt_comment="-umapt"
    else:
        dropping_not_connected = 0
        cr_start()
        kmeans.fit(word_doc_matrix)
        runtime_kmeans = cr_getrt()

    
    stats = {'type': f"kmeans{opt_comment}",
             'testset': t_name,
             'version': td_version,
             'size': clustersize
            }
    
    detail_stats = {'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                'runtime': runtime_kmeans
               }
    
    if('y' in testset.columns):
        detail_stats['umap'] = f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}", #umap not important here
        stats['no_con'] = dropping_not_connected

    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'no_lab':missing_clabel})
    prediction.update(detail_stats)
    res.append(prediction.copy())
        
    display(pd.DataFrame.from_dict(res))
    return kmeans.labels_
    #testset.drop(['cluster_run'], axis=1)


In [30]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = umap_embedding[umap_embedding['y'].notna()]['name'].copy()

for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(umap_embedding, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix



,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,5,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.349354,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,5,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.349354,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
1,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,13,5,13,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.425823,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,5,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.349354,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
1,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,13,5,13,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.425823,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
2,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,16,5,16,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.635159,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,5,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.349354,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
1,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,13,5,13,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.425823,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
2,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,16,5,16,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.635159,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
3,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,19,5,19,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.933086,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,5,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.349354,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
1,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,13,5,13,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.425823,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
2,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,16,5,16,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.635159,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
3,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,19,5,19,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.933086,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
4,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,22,5,22,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",2.353187,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,5,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.349354,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
1,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,13,5,13,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.425823,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
2,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,16,5,16,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.635159,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
3,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,19,5,19,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.933086,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
4,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,22,5,22,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",2.353187,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
5,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,25,5,25,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",2.716186,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,5,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.349354,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
1,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,13,5,13,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.425823,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
2,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,16,5,16,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.635159,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
3,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,19,5,19,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",1.933086,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
4,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,22,5,22,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",2.353187,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
5,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,25,5,25,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",2.716186,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."
6,kmeans-umapt,t2.5.1s+2.5.2+2.5.4+2.5.6,1,28,5,28,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",3.043824,"(hellinger, rt_umap_tf: 12976.100148200989, rt..."


In [31]:
kmeans_labels_results

,name,10,13,16,19,22,25,28
0,technopagan/mss-saliency,5,6,0,9,17,13,21
1,snmnmin12/selective-search,8,7,5,14,8,1,23
2,google/chaps-linux,5,6,3,9,17,4,18
3,lukasolson/speech-router,3,10,11,6,4,18,16
4,XavierTecBr/qbr-admin,5,6,0,4,5,10,11
...,...,...,...,...,...,...,...,...
218373,bhagatabhijeet/bhagatabhijeet,5,2,6,4,16,8,3
218374,LandGrey/SpringBootVulExploit,9,12,13,15,18,9,25
218375,mucomplex/Pentest_checklist,5,6,0,9,5,10,11
218376,bigsen/Vision_Track,0,7,5,14,8,1,23


In [32]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/kmeans-umap_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans-umap_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans-umap_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans-umap_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//kmeans-umap_labels_results_t2.5.1s+2.5.2+2.5.4+2.5.6.pkl


In [33]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = testset['name'].copy()


for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(testset, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",179.437297


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",179.437297
1,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,13,13,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",238.746616


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",179.437297
1,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,13,13,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",238.746616
2,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,16,16,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",391.420928


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",179.437297
1,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,13,13,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",238.746616
2,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,16,16,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",391.420928
3,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,19,19,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",512.098463


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",179.437297
1,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,13,13,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",238.746616
2,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,16,16,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",391.420928
3,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,19,19,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",512.098463
4,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,22,22,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",512.235665


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",179.437297
1,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,13,13,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",238.746616
2,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,16,16,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",391.420928
3,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,19,19,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",512.098463
4,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,22,22,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",512.235665
5,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,25,25,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",514.960050


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,10,10,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",179.437297
1,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,13,13,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",238.746616
2,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,16,16,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",391.420928
3,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,19,19,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",512.098463
4,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,22,22,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",512.235665
5,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,25,25,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",514.960050
6,kmeans,t2.5.1s+2.5.2+2.5.4+2.5.6,1,28,28,"rt_vect: 17.6684889793396, min_df: 4, max_df: ...",712.261844


In [34]:
kmeans_labels_results

,name,10,13,16,19,22,25,28
0,technopagan/mss-saliency,3,9,3,8,18,14,3
1,snmnmin12/selective-search,4,11,13,0,10,0,13
2,google/chaps-linux,9,12,6,6,21,1,1
3,lukasolson/speech-router,2,0,10,3,11,23,12
4,XavierTecBr/qbr-admin,4,5,13,0,10,0,0
...,...,...,...,...,...,...,...,...
218373,bhagatabhijeet/bhagatabhijeet,4,5,2,0,3,3,0
218374,LandGrey/SpringBootVulExploit,8,10,15,9,20,1,19
218375,mucomplex/Pentest_checklist,8,7,5,7,2,15,19
218376,bigsen/Vision_Track,4,5,13,0,10,12,13


In [35]:
print("saving results to disk")

try:
    old_results = from_disk(f'results/{cust_dir}/kmeans_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar1//kmeans_labels_results_t2.5.1s+2.5.2+2.5.4+2.5.6.pkl
